# <span id="up" style="color:red">RED</span>ACTOR

**<span id="up" style="color:red">RED</span>ACTOR is a cozy Juputer Notebook created by [Bilinguator.com](https://bilinguator.com/) to make texts cleaner. It may help to place quotation marks, parentheses, dashes, ellipses in the desired format, remove unnecessary line breaks or page numbers, find and tag titles and character names. For more information, visit [Github page](https://github.com/bilinguator/redactor/).**

|Contents|
|---|
|[Load modules and scripts](#load-modules-and-scripts)|
|[Load text from file](#load-text-from-file)|
|[Technical artifacts](#technical-artifacts)|
|[Alphabet](#alphabet)|
|[Repetitions](#repetitions)|
|[Search for substrings](#search-for-substrings)|
|[Ellipsis](#ellipsis)|
|[Apostrophes](#apostrophes)|
|[Quotes and brackets](#quotes-and-brackets)|
|[Headings](#headings)|
|[Dashes](#dashes)|
|[Characters in the play](#characters-in-the-play)|
|[Dot the ⟨Ё⟩ letters in Russian](#dot-the-yo)|
|[Save text](#save-text)|

# <span id='load-modules-and-scripts'>Load modules and scripts</span>

[Up](#up) | [Save text](#save-text)

In [ ]:
import os
import sys
import math
import html
import re
from tqdm import tqdm
from alphabets import *
from paragraphs import *
from quotes_and_brackets import *
from redactor import *

# <span id='load-text-from-file'>Load text from file</span>

[Up](#up) | [Save text](#save-text)

**`lang` - ISO code of the language. Codes are available in `get_alphabet` function of the `alphabets.py` script.**

**`file_address` - TXT file to be preprocessed.**

**All repeated `\n` are replace by one `\n` here.**

In [ ]:
lang = 'en'
file_address = 'path/to/text'

file = open(file_address, 'r', encoding='utf-8')
text = file.read().strip()
file.close()
text = '\n'.join(re.split('\n+', text))
assert text != '', f'File {file_address} is empty!'
print(f'File \033[1m{file_address}\033[0m contains:')
print(f'• {len(text)} symbols;')
print('•', len(text.split('\n')), 'paragraphs.')

**Convert special HTML characters to the readable Unicode characters.**

In [ ]:
text = html.unescape(text)

# <span id='technical-artifacts'>Technical artifacts</span>

[Up](#up) | [Save text](#save-text)

**Remove technical line breaks. By technical line breaks, we mean the breaks formed in the process of translating text from one file extension to another. So, when saving a TXT file from PDF, there are many line breaks in the middle of sentences. The `remove_tech_line_breaks` function removes such breaks and returns a repaired text.**

In [ ]:
print(len(text.split('\n')), 'paragraphs before removing technical line breaks.')
text = remove_tech_line_breaks(text, line_ends=get_quotes() | set('.!?;-–—‒'))
print(len(text.split('\n')), 'paragraphs after removing technical line breaks.')

**Detect paragraphs with page numbers.**

In [ ]:
page_number_paragraphs = detect_chapters(text, chapter='', numbering='arabic')
print('Paragraphs with page numbers:', page_number_paragraphs)

**Remove paragraphs with page numbers.**

In [ ]:
text = remove_paragraphs(text, page_number_paragraphs)

**Detect page headings and other repeating paragraphs.**

In [ ]:
repeated_paragraphs = count_repeated_paragraphs(text)

if bool(repeated_paragraphs):
    print('Repeating paragraphs\n(times)')
    for key, value in repeated_paragraphs.items():
        print(f'\n{key}\n({value})')
else:
    print('No repeating paragraphs found.')

**Specify repeating paragraph to be removed in the `paragraph_to_remove` variable.**

In [ ]:
paragraph_to_remove = 'Спасибо, что скачали книгу в бесплатной электронной библиотеке Royallib.ru'

text = re.split(rf'{paragraph_to_remove}$', text, flags=re.MULTILINE)
text = [c.strip() for c in text]
text = '\n'.join(text)

**Find footnote links like `{1}`, `[1]`, `(1)`, etc.**

In [ ]:
footnote_regex = '{\d+}|\[\d+\]|\(\d+\)'
footnote_links = set(re.findall(footnote_regex, text))
if bool(footnote_links):
    print(*footnote_links)
else:
    print('No footnote links found.')

**Remove footnote links.**

In [ ]:
text = ''.join(re.split(footnote_regex, text))

# <span id='alphabet'>Alphabet</span>

[Up](#up) | [Save text](#save-text)

**Check text’s alphabet.**

In [ ]:
text_characters = set(text.upper()) - set([' ', '\n'])
print('Unique characters of the text:')
print(set_to_str(text_characters))
print()
print(f'Characters of the text subtracted by the \033[1m{lang}\033[0m alphabet:')
print(set_to_str(text_characters - get_alphabet(lang)))
print()
print(f'\033[1m{lang}\033[0m alphabet:')
print(set_to_str(get_alphabet(lang)))
print()
print(f'Unused characters of the \033[1m{lang}\033[0m alphabet:')
print(set_to_str(get_alphabet(lang) - text_characters))

# <span id='repetitions'>Repetitions</span>

[Up](#up) | [Save text](#save-text)

**Find characters repetitions in the text.**

In [ ]:
print('Characters’ repetitions in the text:')
text_characters_all = set(text.upper())
text_upper = text.upper()
for c in text_characters_all:
    count = text_upper.count(c*2)
    if count > 0:
        print(f'{c*2} - {count}')

**Specify repetition in the `query` variable and explore it in the text. If `query`’s case matters, specify it in the `case_matters` attribute of the `print_paragraphs_by_query` function.**

In [ ]:
query = '--'
print_paragraphs_by_query(text, query, case_matters=False)

# <span id='search-for-substrings'>Search for substrings</span>

[Up](#up) | [Save text](#save-text)

**Type a wanted character to `query` variable and count how many hits are found in particular paragraphs. If `query`’s case matters, specify it in the `case_matters` attribute of the `print_paragraphs_by_query` function.**

In [ ]:
query = '-'
search_in_paragraphs(text, query, case_matters=False)

**Print paragraphs with the `query`. If `query`’s case matters, specify it in the `case_matters` attribute of the `print_paragraphs_by_query` function.**

In [ ]:
query = '('
print_paragraphs_by_query(text, query, case_matters=False)

# <span id='ellipsis'>Ellipsis</span>

[Up](#up) | [Save text](#save-text)

**Replace three dots to ellipsis signs. Explore the three dots separated by spaces `. . .` (common for English and other texts) and regular `...`.**

In [ ]:
print_paragraphs_by_query(text, '. . .', case_matters=False)

In [ ]:
print_paragraphs_by_query(text, '...', case_matters=False)

In [ ]:
print_paragraphs_by_query(text, '…', case_matters=False)

**Replace to ellipsis sign `…`.**

**`. . . .` → `. …`**

In [ ]:
old = '. . . .'
new = '. …'
print(text.count(old), '- before replacement.')
text = text.replace(old, new)
print(text.count(old), '- after replacement.')

**`. . .` → `…`**

In [ ]:
old = '. . .'
new = '…'
print(text.count(old), '- before replacement.')
text = text.replace(old, new)
print(text.count(old), '- after replacement.')

**`...` → `…`**

In [ ]:
old = '...'
new = '…'
print(text.count(old), '- before replacement.')
text = text.replace(old, new)
print(text.count(old), '- after replacement.')

# <span id='apostrophes'>Apostrophes</span>

[Up](#up) | [Save text](#save-text)

**Count all apostrophe characters in the text before recovering.**

In [ ]:
print(text.count('\''), '- typewriter apostrophes ⟨\'⟩.')
print(text.count('’'), '- punctuation apostrophes ⟨’⟩.')

**Replace typewriter apostrophe `'` to punctuation apostrophe `’`. To replace all apostrophes located inside words, specify `replace_inner=True`.**

In [ ]:
text = recover_apostrophes_by_lang(text, lang, replace_inner=False)

**Count all apostrophe characters in the text after recovering.**

In [ ]:
print(text.count('\''), '- typewriter apostrophes ⟨\'⟩.')
print(text.count('’'), '- punctuation apostrophes ⟨’⟩.')

**Explore the paragraphs with punctuation apostrophes ⟨`’`⟩.**

In [ ]:
print_paragraphs_by_query(text, '’')

**Explore the paragraphs with typewriter apostrophes ⟨`'`⟩.**

In [ ]:
print_paragraphs_by_query(text, '\'')

**Replace typewriter apostrophes ⟨`'`⟩ with punctuation apostrophes ⟨`’`⟩ in interaction mode. Leave input fields empty if no replacement is needed, otherwise type any characters.**

In [ ]:
text = replace_interactively(text, '\'', '’', scope=55)

In [ ]:
print_paragraphs_by_query(text, '\'')

# <span id='quotes-and-brackets'>Quotes and brackets</span>

[Up](#up) | [Save text](#save-text)

**What quotation marks are there in the text?**

In [ ]:
text_characters = set(text.upper()) - set([' ', '\n'])

print('Quotation marks found in the text:')
text_quotes = set_to_str(text_characters & get_quotes())
print(text_quotes)
print()

print(f'Quotation marks of the \033[1m{lang}\033[0m language:')
print(quotes_to_str_by_lang(lang))
print()

print(f'Quotation marks of the \033[1m{lang}\033[0m language used in the text:')
print(set_to_str(set(text_quotes) & set(quotes_to_str_by_lang(lang))))

**Choose target quotation marks of the language of interest. Then, replace quotation marks in the interaction mode:**

**`old_quote` - attribute for quotation mark (opening or closing) to be replaced;**

**`new_quote` - attribute for target quotation mark.**

In [ ]:
get_quotes(lang)

In [ ]:
old_quote = '"'
new_quote = '“'
text = replace_interactively(text, '"', '“')

# <span id='headings'>Headings</span>

[Up](#up) | [Save text](#save-text)

**Detect paragraphs containing chapters’ headings. Specify arguments in the `detect_chapters` function:**

**`chapter` - string containing key word for a chapter, e.g. “Chapter” for English, “Глава” for Russian, “Chapitre” for French, etc.; word case matters here; the space character delimiting the key word from a numeral is the part of this argument;**

**`numbering` - string of one of the following variants:**
* **`"arabic"` for Arabic numbers (1, 2, 3, etc.);**
* **`"roman"` for the Roman numbers (I, II, III, IV, etc.);**
* **`"ja"` or `"zh"` for Japanese or Chinese numerals (一、二、三、四, etc.);**
* **`"text"` for non-numeric characters; suitable if numerals in words are presented (‘One’, ‘Two’, ‘Three’, etc.)**

**`delimiter` - delimiter among the chapter-numbering and title parts;**

**`with_title` - boolean specifying if chapters have titles;**

**`numbering_first` - boolean specifying if numbering precedes chapter key word (‘XIX chapter’).**

**Example**

If the chapters’ headings of your text look like `Chapter MMMCMXCIX — Epilogue`, specify arguments as follows:

`chapter = "Chapter "`

`numbering = "roman"`

`delimiter = " — "`

`with_title = True`

`numbering_first = False`

In [ ]:
print('Chapters detected:\n')
chapter_paragraphs = detect_chapters(text, chapter='', numbering='roman', delimiter='',
                                     with_title=False, numbering_first=True)
print('\nTotal count:', len(chapter_paragraphs))
print('Paragraphs set:', chapter_paragraphs, sep='\n')

**If there are paragraph numbers to be added or removed from the set, specify them in the `added_paragraphs` and `removed_paragraphs` variables respectively.**

In [ ]:
added_paragraphs = set([])
removed_paragraphs = set([])
chapter_paragraphs = (chapter_paragraphs | added_paragraphs) - removed_paragraphs
print('Paragraphs set:', *sorted(list(chapter_paragraphs)), sep='\n')

**Enclose the detected chapters’ headings in `<h1></h1>` tags.**

In [ ]:
text = tag_paragraphs(text, chapter_paragraphs, tag='h1')

# <span id='dashes'>Dashes</span>

[Up](#up) | [Save text](#save-text)

**`-` — hyphen, part of compound words;**

**`–` — en dash, used for number intervals;**

**`—` — em dash, a multifunctional punctuation mark for dialogues, thought abruptions, instead of colons and parenthetical marks, etc.**

**Detect if there are dialogue paragraphs and what dash they start with.**

In [ ]:
hyphen = '-'
en_dash = '–'
em_dash = '—'

print(f'Paragraphs starting with hyphen ⟨{hyphen}⟩:')
print(search_paragraphs_by_regex(text, f'^{hyphen}.*'))

print(f'Paragraphs starting with en dash ⟨{en_dash}⟩:')
print(search_paragraphs_by_regex(text, f'^{en_dash}.*'))

print(f'Paragraphs starting with em dash ⟨{em_dash}⟩:')
print(search_paragraphs_by_regex(text, f'^{em_dash}.*'))

**Print paragraphs starting with the dash of interest. Specify the dash of interest in the `dash_of_interest` variable.**

In [ ]:
dash_of_interest = em_dash
print_paragraphs_by_regex(text, f'^{dash_of_interest}.*')

**Replace dash for the dialogue paragraphs to the appropriate one.**

* **`old_dash` — dash symbol to be replaced;**
* **`new_dash` — new dash symbol of interest.**

In [ ]:
old_dash = en_dash
new_dash = em_dash

text = replace_by_regex(text, f'(^|\n)+{old_dash}.?', old_dash, new_dash)

print(f'Paragraphs starting with the dash of interest ⟨{new_dash}⟩:')
print(search_paragraphs_by_regex(text, f'^{new_dash}.*'))

**Print paragraphs with the new dashes.**

In [ ]:
dash_of_interest = em_dash
print_paragraphs_by_regex(text, f'^{dash_of_interest}.*')

**Find all number intervals with dahses.**

In [ ]:
print(f'Number intervals with hyphen ⟨{hyphen}⟩:')
hyphen_intervals = get_number_intervals_by_dash(text, hyphen)
print(hyphen_intervals)

print(f'Number intervals with en dash ⟨{en_dash}⟩:')
en_dash_intervals = get_number_intervals_by_dash(text, en_dash)
print(en_dash_intervals)

print(f'Number intervals with em dash ⟨{em_dash}⟩:')
em_dash_intervals = get_number_intervals_by_dash(text, em_dash)
print(em_dash_intervals)

**Replace all number intervals with hyphen and em dash to intervals with en dash.**

In [ ]:
text = replace_by_regex(text, f'\d+ ?{hyphen} ?\d+', hyphen, en_dash)

In [ ]:
text = replace_by_regex(text, f'\d+ ?{hyphen} ?\d+', em_dash, en_dash)

**Remove space characters in the number intervals. Example: `123 – 456` → `123–456`.**

In [ ]:
text = replace_by_regex(text, f'\d+ {en_dash} \d+', ' ', '')

**Check all number intervals with dahses.**

In [ ]:
print(f'Number intervals with hyphen ⟨{hyphen}⟩:')
hyphen_intervals = get_number_intervals_by_dash(text, hyphen)
print(hyphen_intervals)

print(f'Number intervals with en dash ⟨{en_dash}⟩:')
en_dash_intervals = get_number_intervals_by_dash(text, en_dash)
print(en_dash_intervals)

print(f'Number intervals with em dash ⟨{em_dash}⟩:')
em_dash_intervals = get_number_intervals_by_dash(text, em_dash)
print(em_dash_intervals)

**Find hyphens surrounded by spaces.**

In [ ]:
set(re.findall(f'\S+ {hyphen} [\S]+', text))

**Replace hyphens surrounded by spaces to em dashes. Repeat the previous step while new hyphens are found.**

In [ ]:
text = replace_by_regex(text, f'\S+ {hyphen} \S+', hyphen, em_dash)

**Find en dashes surrounded by spaces.**

In [ ]:
set(re.findall(f'\S+ {en_dash} \S+', text))

**Replace en dashes surrounded by spaces to em dashes. Repeat the previous step while new en dashes are found.**

In [ ]:
text = replace_by_regex(text, f'\S+ {en_dash} \S+', en_dash, em_dash)

**Search for em dashes with spaces on one or both sides.**

In [ ]:
print(set(re.findall(f'\S+{em_dash} | {em_dash}\S+| {em_dash} ', text)))

**Print paragraphs containing suspected mistakes.**

In [ ]:
query = ' -'
print_paragraphs_by_query(text, query, case_matters=False)

# <span id='characters-in-the-play'>Characters in the play</span>

[Up](#up) | [Save text](#save-text)

**Paragraphs with speaking characters start with their names. The names should be enclosed in `<b></b>` tags.**

**List all the characters of the play in the `characters` list variable.**

In [ ]:
characters = [
    'Greg',
    'Samp',
    'Abr',
    'Tyb',
    'Ben',
    'Mont',
    'Prince',
    'Romeo',
    'Juliet',
    'Friar L'
]

**Print all the dialogue paragraphs with their numbers. `dialogue_delimiter` specifies what punctuation mark separates characters’ names from their speeches. Use backslash `\` before symbols reserved for regular expressions.**

In [ ]:
dialogue_delimiter = '\.'
characters_regex = '|'.join(characters)
characters_regex = f'^({characters_regex}){dialogue_delimiter}.*'
print(f'Regular expression to search for dialogues:\n{characters_regex}\n')
print_paragraphs_by_regex(text, characters_regex)

**Get dialogue paragraphs numbers. Change the `dialogue_paragraphs` variable if some of paragraphs do not suit.**

In [ ]:
print('Numbers of dialogue paragraphs:')
dialogue_paragraphs = search_paragraphs_by_regex(text, characters_regex)
print(dialogue_paragraphs)

**Enclose characters’ names in `<b><\b>` tags.**

In [ ]:
text = tag_characters(text, dialogue_paragraphs, characters, '.', tag='b')

**Merge speeches of characters with `<delimiter>` if this is poetry.**

**NB! The `merge_speeches` function merges all the paragraphs not containing `<b>` and `<h1>` tags! These paragraphs may not be parts of speeches.**

In [ ]:
text = merge_speeches(text)
print('Current statistics:')
print(f'• {len(text)} symbols;')
print('•', len(text.split('\n')), 'paragraphs.')

# <span id='dot-the-yo'>Dot the ⟨Ё⟩ letters in Russian</span>

[Up](#up) | [Save text](#save-text)

**Use Ёditor to dot the ⟨Ё⟩ letters in a Russian text. For more information about the instrument visit its [Github page](https://github.com/bilinguator/yoditor).**

**For a start, specify the location of the `yoditor` directory with all Ёditor contents in the `yoditor_dir_path` variable. If it is located in the same directory as `redactor`, leave `yoditor_dir_path` equal `'..'`**

In [ ]:
yoditor_dir_path = '..'
project_dir = os.path.abspath(yoditor_dir_path)
sys.path.insert(0, project_dir)
import yoditor.yoditor as yoditor

**Dot the ⟨Ё⟩ letters in the words which always spelled with them.**

In [ ]:
text = yoditor.recover_yo_sure(text)

**Dot the ⟨Ё⟩ letters in the words which are ambiguous about its spelling. Do it in the interactive mode. For every word, the input field is shown. If the replacement is needed type the `ё` letter in the field, otherwise leave it empty. Press Enter to confirm your choise.**

In [ ]:
text = yoditor.recover_yo_unsure(text)

**Find paragraphs with words of ambiguous spelling. If `query`’s case matters, specify it in the `case_matters` attribute of the `print_paragraphs_by_query` function.**

In [ ]:
query = 'вперёд'
print_paragraphs_by_query(text, query, case_matters=False)

# <span id='save-text'>Save text</span>

[Up](#up)

**Specify another file address in the `file_address` variable if needed.**

In [ ]:
file_address = file_address
file = open(file_address, 'w', encoding='utf-8')
file.write(text)
file.close()